### Model

In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Conv2D
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import cv2
import ntpath

In [4]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!git clone https://github.com/tzudir/Self_driving

Cloning into 'Self_driving'...
remote: Enumerating objects: 9627, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9627 (delta 0), reused 10 (delta 0), pack-reused 9617
Receiving objects: 100% (9627/9627), 104.25 MiB | 14.37 MiB/s, done.
Updating files: 100% (9618/9618), done.


In [2]:
data_url = 'https://github.com/tzudir/Self_driving/blob/main/data/driving_log.csv'
data = pd.read_csv(data_url)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 41, saw 13
